In [96]:
import re
import nltk
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
%matplotlib inline

In [75]:
df = pd.read_csv('august_data.csv')
df.Description.head()

0    Chat transcript\n\n\t\t\tE-mail: bistro@tohu.c...
1    Chat transcript\n\n\t\t\tName: Rebekkah Damian...
2    Chat transcript\n\n\t\t\tE-mail: jaycen@me.com...
3    Chat transcript\n\n\t\t\tE-mail: mario@elevati...
4    Chat transcript\n\n\t\t\tE-mail: opnaples@came...
Name: Description, dtype: object

In [80]:
data = df.Description.values.tolist()
data = [re.sub('\S*@\S*\s?', "", sent) for sent in data]
data = [re.sub('\s+', " ", sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]
data = [re.sub('https:\S+',"", sent) for sent in data]

pprint(data[:1])

['Chat transcript E-mail: Victoria Fri, 08/27/21 12:54:54 pm America/Toronto '
 'BevSpot Support here! How can we help? Bistro Tohu 12:55:16 pm hi i would '
 'like to import invoices can you activate the addon? Victoria 12:56:14 pm '
 'Hello - Thank you for reaching out! Are you referring to invoice processing? '
 'Bistro Tohu 12:56:25 pm i guess i want to import csv and pdf for creating '
 'items Victoria 01:02:13 pm Ok... Just to clarify, these are going to be '
 'invoiced from a vendor, and thats what youre looking to enter into the '
 'account? Or do you have a list of items on a file that we can upload through '
 'the items section? Just double-checking because currently, invoice '
 'processing is an add-on feature that is not included within the freemium '
 'program. Bistro Tohu 01:02:55 pm from invoice i m ok to pay Victoria '
 '01:04:12 pm Ok, thank you for this information. Hang tight for a quick '
 'moment while I reach out to your account manager. Bistro Tohu 01:04:52 pm '
 

In [93]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))

In [83]:
bigram = gensim.models.Phrases(data, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [86]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [94]:
data_words_nostops = remove_stopwords(data)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [88]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

In [89]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:10]]

[[('access', 3),
  ('account', 4),
  ('accounting', 1),
  ('activate', 1),
  ('add', 1),
  ('allow', 1),
  ('annually', 1),
  ('assist', 1),
  ('bevspot', 1),
  ('bill', 1),
  ('bistro', 5),
  ('chat', 2),
  ('check', 1),
  ('clarify', 1),
  ('cog', 1),
  ('colleague', 1),
  ('come', 1),
  ('couple', 1),
  ('create', 1),
  ('csv', 1),
  ('currently', 2),
  ('datum', 1),
  ('double', 1),
  ('enough', 1),
  ('excess', 1),
  ('export', 1),
  ('feature', 3),
  ('feel', 1),
  ('file', 1),
  ('free', 1),
  ('freemium', 1),
  ('good', 2),
  ('guess', 1),
  ('guy', 1),
  ('hang', 1),
  ('historical', 1),
  ('import', 3),
  ('include', 1),
  ('information', 1),
  ('initial', 1),
  ('inventory', 1),
  ('invoice', 8),
  ('item', 3),
  ('list', 1),
  ('look', 1),
  ('mail', 1),
  ('manager', 3),
  ('marguerite', 1),
  ('moment', 1),
  ('month', 1),
  ('next', 1),
  ('open', 1),
  ('order', 1),
  ('pay', 1),
  ('pdf', 1),
  ('pm', 3),
  ('processing', 3),
  ('program', 2),
  ('quick', 1),
  ('re', 

In [90]:
#no touch
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [95]:
#score
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.131765481511145

Coherence Score:  0.4952112186344836


In [92]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.159791  0.061412       1        1  24.898993
15     0.143695 -0.211979       2        1  14.576391
18    -0.266080 -0.102626       3        1  13.925148
19    -0.035589  0.239413       4        1   9.168967
2      0.206177 -0.127120       5        1   8.119560
16     0.241781 -0.005833       6        1   6.710317
13     0.065023  0.215375       7        1   6.669054
1     -0.073614 -0.267491       8        1   6.033274
9     -0.181675  0.169304       9        1   5.257285
7      0.219480  0.150353      10        1   4.361059
11    -0.046401 -0.011899      11        1   0.030448
4     -0.047250 -0.012437      12        1   0.029708
17    -0.047559 -0.011416      13        1   0.029647
5     -0.047865 -0.012036      14        1   0.028128
6     -0.047995 -0.011846      15        1   0.027652
3     -0.048496 -0.012271      16        1   0.027620
0     -0.048055 -0.012197      17        1   0.027032
12    -0.048456 -0.012233      18        1   0.026906
14    -0.048457 -0.012235      19        1   0.026404
10    -0.048457 -0.012235      20        1   0.026404, topic_info=        Term       Freq      Total Category  logprob  loglift
52     order  28.000000  28.000000  Default  30.0000  30.0000
41   invoice  39.000000  39.000000  Default  29.0000  29.0000
55        pm  84.000000  84.000000  Default  28.0000  28.0000
121     send  32.000000  32.000000  Default  27.0000  27.0000
62    report  35.000000  35.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
25    export   0.001129   1.333837  Topic20  -6.4184   1.1645
26   feature   0.001129   4.829839  Topic20  -6.4184  -0.1222
27      feel   0.001129   4.976739  Topic20  -6.4184  -0.1522
28      file   0.001129   7.586928  Topic20  -6.4184  -0.5739
29      free   0.001129   4.977081  Topic20  -6.4184  -0.1523

[1343 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         1  0.216483     access
0         4  0.324724     access
0         7  0.432966     access
1         1  0.093443    account
1         2  0.140165    account
...     ...       ...        ...
330       2  0.728100      wrong
135       6  0.473613        www
135       9  0.473613        www
373       1  0.919246  yesterday
516       1  0.720866    zachary

[957 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 16, 19, 20, 3, 17, 14, 2, 10, 8, 12, 5, 18, 6, 7, 4, 1, 13, 15, 11])